In [ ]:
import numpy as np
import nibabel as nib
import math
import os
from skimage.util import view_as_windows
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import cv2
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#creating a list with sorted files in nifti format from the dataset location
data_path = "/content/drive/My Drive/Colab Notebooks/BRATS/HGG"
list_dir = os.listdir(data_path)
list_dir_sorted = sorted(list_dir)

In [ ]:
#read the data in nift format and return numpy array 
def data_loader(x):
  for subject in os.listdir(data_path+list_dir_sorted[x]):
    if subject[-8] == str(1):
      filename=os.path.join(data_path+list_dir_sorted[x],subject)
      #print(subject)
      proxy = nib.load(filename)
      data = np.array(proxy.dataobj, order='C')
      data = np.einsum('kli->ikl', data)
  return data

In [ ]:
def replace_value(slce_ds, slce):
  max_org = slce.max()
  min_org = slce.min()
  max = slce_ds.max()
  min = slce_ds.min()
  for i in range(64):
    for j in range(64):
      if slce_ds[i][j]==max:
        slce_ds[i][j]=max_org
      elif slce_ds[i][j]==min:
        slce_ds[i][j]=min_org
  return slce_ds

In [ ]:
def getLR(hr_data):
    imgfft = np.fft.fftn(hr_data)
    x_center = imgfft.shape[0] // 2
    y_center = imgfft.shape[1] // 2
    imgfft[x_center-100 : x_center+100, y_center-100 : y_center+100] = 0
    imgifft = np.fft.ifftn(imgfft)
    img_out = abs(imgifft)
    return img_out

In [ ]:
#saves the HR and LR images in numpy format from nift format to be used for testing and training Real-ESRGAN and ESRGAN
#specify the location to save the HR and LR images
dim = (64,64)
count_neg = 0
a_ssim = []
start = 0
end = 223
count = 0
count_blank = 0

for i in range(start,end):
  data = data_loader(i)
  data = data.astype('float32')
  print(data.shape)
  for j,img in enumerate(data):
    count+=1
    max = np.max(img)
    min = np.min(img)
    if max != min:
      count_blank+=1
      img = (img-np.min(img))/(np.max(img)-np.min(img))
      a = int((256-img.shape[0])/2)
      b = int((256-img.shape[1])/2)
      if a<0 or b<0:
        count_neg+=1
        continue
      img = np.pad(img, [(a, a), (b, b)], mode='constant')
      #img = np.rot90(img)
      img = img[:,:,np.newaxis]
      np.save(str(i) + "_slc" + str(j)  + "_org.npy", img)

      slce_ds = getLR(img)
      slce_ds = cv2.resize(slce_ds, dim, interpolation=cv2.INTER_LINEAR)
      slce_ds = replace_value(slce_ds, img)
      slce_ds =(slce_ds-np.min(slce_ds))/(np.max(slce_ds)-np.min(slce_ds))
      np.save(str(i) + "_slc" + str(j)  + "_lr.npy", slce_ds)  